In [0]:
import time
import copy
import hashlib
from operator import add
from functools import reduce
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sc=spark.sparkContext

In [0]:
zeroVal1 = ([], 0)
mergeVal1 = (lambda aggregated, el: (aggregated[0] + [el[0]], aggregated[1] + el[1]))    
mergeComb1 = (lambda agg1,agg2:agg1+agg2)
def onebyte_hash(s):
    return int(hashlib.sha1(s.encode("utf-8")).hexdigest(), 16) % (10 ** 8)

In [0]:
def generate_computational_graph(RHS, schema):
    """
    Output
    ----------
    A dictionary where
    key: level
    value: list of current level's candidates, candidates are in the format of set
    -----

    """
    computational_graph=dict()
    for level in range(3):
        #use brute force to generate candidates for each level
        computational_graph[level]=[]
        if level== 0:
            for attribute  in schema:
                if attribute !=RHS:
                    computational_graph[level].append(set([attribute]))

        else:
            for element1 in computational_graph[level-1]:
                for element2 in computational_graph[0]:
                    newelement = element1.union(element2)
                    if newelement not in computational_graph[level]:
                        if len(newelement)==level+1:
                            computational_graph[level].append(newelement)    

    return computational_graph
def get_candidates(level, computational_graph):
    return computational_graph[level]
def prune_graph(level,current_level_result,computational_graph):
    """
    Input
    -------
    current_level_result: (soft/delta) functional dependencies discovered by algorithm, data structure: a list of candidates where candidates are in the format of sets
    computational_graph: A dict where key:level value: list of current level's candidates, candidates are in the format of set

    Output
    -------
    A pruned computational graph
    """
    # Candidates are pruned because minimal FD are already discovered

    # prune candidates after this level by verifying whether the next level has previous level's candidates as subset
    new_computational_graph = copy.deepcopy(computational_graph)
    while level<2:
        level+=1
        for LHS in current_level_result:
            for candidate in computational_graph[level]:
                if LHS.issubset(candidate):
                    if candidate in new_computational_graph[level]:
                        new_computational_graph[level].remove(candidate)


    return new_computational_graph
def transform_res(FDs):
    """
    Parameters
    --------------
    FDs: a list of (soft/delta) functional dependencies, where elements are tuples(LHS,RHS), LHS is in the format of set

    Output
    ---------
    current_level_result: a dictionary where key: RHS value: a list of LHS where candidates are in the form of sets
    """

    current_level_result=dict()
    for (LHS,RHS) in FDs:
        if RHS not in current_level_result.keys():
            current_level_result[RHS]=[]

        current_level_result[RHS].append(LHS)

    return current_level_result

In [0]:
def onebyte_hash(s):
    return int(hashlib.sha1(s.encode("utf-8")).hexdigest(), 16) % (10 ** 8)

In [0]:
def find_softFDs_pairs(level, df,current_level_candidates):
  softFDs = []
  candidates_num = 0
  for RHS in current_level_candidates.keys():
      for LHS in current_level_candidates[RHS]:
        candidates_num += 1
  i = 0
  array_rdds = [spark.sparkContext.emptyRDD()] * candidates_num
  for RHS in current_level_candidates.keys():
      for LHS in current_level_candidates[RHS]:
          rddt=df.rdd.map(lambda x: (*LHS, RHS, tuple(x[idx] for idx in list(map(lambda y: schema.index(y),LHS))), x[schema.index(RHS)]))
          array_rdds[i] = array_rdds[i].union(rddt)
          i += 1

  if len(array_rdds) >= 1:
    rdds = sc.union(array_rdds)
  else:
    rdds = spark.sparkContext.emptyRDD()
  rdds = rdds.map(lambda x: (x,1))\
            .coalesce(4)\
            .reduceByKey(add)\
            .map(lambda x: ((x[0][:-1]), (x[1], x[1])))\
            .partitionBy(4, lambda tup: onebyte_hash(''.join(tup[0])))\
            .aggregateByKey(zeroVal1,mergeVal1,mergeComb1)\
            .map(lambda x: ((x[0][:-1]), list(map(lambda r: r / x[1][1],x[1][0]))))\
            .map(lambda x: (x[0], any(p >= 0.8 for p in x[1])))\
            .coalesce(4)\
            .reduceByKey(lambda x, y: x * y)\
            .filter(lambda x: x[1] == 1)\
            .map(lambda x:(*x[0][:level],x[0][level]))\
            .distinct()\
            .collect()
            
            
  for item in rdds:
      softFDs.append(({*item[:-1]},item[-1]))

  return softFDs

In [0]:
def controller(df, func):
  """
  A control flow function

  Parameters
  -----------
  func: (soft/delta) Functional Discovery functions
  df: dataframe

  Output
  ------
  (soft/delta) Functional Dependencies
  """  
  # Initialization: Generate computational graph for each attribute which will be on RHS
  schema = df.columns
  computational_graph=dict()
  FDs=[]
  for RHS in schema:
    computational_graph[RHS]=generate_computational_graph(RHS,schema)

  for level in range(3):
    # Get current level candidates
    current_level_candidates=dict()
    for RHS in computational_graph.keys():
      current_level_candidates[RHS] = get_candidates(level,computational_graph[RHS])

    # Use current_level candidates as an input to FD-functions for each level, func will return discovered (soft/delta)functional dependencies
    tFDs = func(level,df,current_level_candidates)
    FDs.extend(tFDs)
    #Transform res into a dictionary where key: RHS value: a list of LHS where candidates are in the form of sets
    current_level_result = transform_res(tFDs)

    # Prune graphs according to feedback of FD-functions
    for RHS in computational_graph.keys():
      if RHS in current_level_result.keys():
        computational_graph[RHS]=prune_graph(level, current_level_result[RHS],computational_graph[RHS])


  return FDs

In [0]:
computational_graph=dict()
df_10000 = spark.sql("SELECT * FROM _2018_01_bme280sof_3_csv limit 10000")
df_10000=df_10000.drop('_c0')
schema = df_10000.columns

In [0]:
start_time = time.time()
# softFDs = find_softFDs_pairs(1, df_10000, current_level_candidates)
softFDs = controller(df_10000, find_softFDs_pairs)
print("--- %s seconds ---" % (time.time() - start_time))
print(softFDs)

--- 192.77763414382935 seconds ---
[(set(), 'location'), (set(), 'sensor_id'), ({'humidity'}, 'pressure'), ({'timestamp'}, 'pressure'), ({'location'}, 'temperature'), ({'location'}, 'lat'), ({'sensor_id'}, 'timestamp'), ({'timestamp'}, 'location'), ({'sensor_id'}, 'humidity'), ({'humidity'}, 'location'), ({'location'}, 'pressure'), ({'lon'}, 'sensor_id'), ({'sensor_id'}, 'location'), ({'sensor_id'}, 'temperature'), ({'sensor_id'}, 'lat'), ({'sensor_id'}, 'pressure'), ({'lon'}, 'location'), ({'lon', 'timestamp'}, 'lat'), ({'lon', 'timestamp'}, 'temperature'), ({'lon', 'temperature'}, 'pressure'), ({'lon', 'humidity'}, 'temperature'), ({'timestamp', 'temperature'}, 'humidity'), ({'sensor_id', 'timestamp'}, 'humidity'), ({'timestamp', 'location'}, 'humidity'), ({'lat', 'location'}, 'temperature'), ({'lon', 'timestamp'}, 'location'), ({'timestamp', 'lat'}, 'location'), ({'lon', 'timestamp'}, 'pressure'), ({'timestamp', 'location'}, 'lat'), ({'lat', 'location'}, 'pressure'), ({'humidity', 'location'}, 'lat'), ({'lon', 'humidity'}, 'pressure'), ({'timestamp', 'location'}, 'pressure'), ({'timestamp', 'lat'}, 'pressure'), ({'timestamp', 'temperature'}, 'pressure'), ({'humidity', 'temperature'}, 'pressure'), ({'sensor_id', 'location'}, 'temperature'), ({'sensor_id', 'lat'}, 'temperature'), ({'sensor_id', 'timestamp'}, 'pressure'), ({'sensor_id', 'humidity'}, 'pressure'), ({'humidity', 'location'}, 'pressure'), ({'humidity', 'lat'}, 'pressure'), ({'sensor_id', 'location'}, 'lat'), ({'sensor_id', 'timestamp'}, 'location'), ({'sensor_id', 'location'}, 'pressure'), ({'sensor_id', 'humidity'}, 'location'), ({'sensor_id', 'timestamp'}, 'lat'), ({'sensor_id', 'lat'}, 'pressure'), ({'sensor_id', 'humidity'}, 'lat'), ({'sensor_id', 'timestamp'}, 'temperature'), ({'sensor_id', 'temperature'}, 'pressure'), ({'sensor_id', 'humidity'}, 'temperature'), ({'humidity', 'location'}, 'temperature'), ({'humidity', 'lat'}, 'temperature'), ({'lat', 'pressure'}, 'temperature'), ({'timestamp', 'location'}, 'temperature'), ({'timestamp', 'lat'}, 'temperature'), ({'timestamp', 'humidity'}, 'pressure'), ({'location', 'pressure'}, 'temperature'), ({'lon', 'timestamp'}, 'humidity')]